In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 21:33 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 21:33 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal *.csv  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/tbl0.csv': File exists
copyFromLocal: `/tmp/wordcount/tbl1.csv': File exists


In [4]:
%%hive

DROP TABLE IF EXISTS tbl0;
DROP TABLE IF EXISTS tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 13.734 seconds
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.191 seconds


In [5]:
%%hive

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.947 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.54 seconds


In [6]:
%%hive

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.098 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 0.979 seconds


In [7]:
%%hive
SELECT c1, c2
FROM tbl0;

SELECT c1, c2
FROM tbl0;
OK
1	D
2	C
3	D
4	D
5	C
6	A
7	B
8	C
9	B
10	B
Time taken: 1.78 seconds, Fetched: 10 row(s)


In [11]:
%%hive
SELECT c1, k, v
FROM tbl1
LATERAL VIEW explode(c4) tbl1 AS k, v;

SELECT c1, k, v
FROM tbl1
LATERAL VIEW explode(c4) tbl1 AS k, v;
OK
1	A	0
1	B	4
1	C	1
1	D	3
1	E	5
2	A	4
2	B	1
2	C	0
2	D	5
2	E	2
3	A	5
3	B	4
3	C	3
3	D	1
3	E	0
4	A	2
4	B	4
4	C	3
4	D	5
4	E	1
5	A	4
5	B	0
5	C	2
5	D	5
5	E	3
6	A	4
6	B	5
6	C	1
6	D	2
6	E	3
7	A	0
7	B	5
7	C	2
7	D	4
7	E	3
8	A	4
8	B	2
8	C	5
8	D	3
8	E	0
9	A	1
9	B	4
9	C	2
9	D	5
9	E	3
10	A	2
10	B	3
10	C	4
10	D	1
10	E	0
Time taken: 0.068 seconds, Fetched: 50 row(s)


In [15]:
%%hive
DROP TABLE IF EXISTS joined_kv;
CREATE TABLE joined_kv AS
(SELECT a.c1, a.c2, b.v
FROM
(SELECT c1, c2
FROM tbl0) a LEFT JOIN
(SELECT c1, k, v
FROM tbl1
LATERAL VIEW explode(c4) tbl1 AS k, v) b
ON (a.c1 = b.c1 AND a.c2 = b.k));

DROP TABLE IF EXISTS joined_kv;
OK
Time taken: 0.013 seconds
CREATE TABLE joined_kv AS
(SELECT a.c1, a.c2, b.v
FROM
(SELECT c1, c2
FROM tbl0) a LEFT JOIN
(SELECT c1, k, v
FROM tbl1
LATERAL VIEW explode(c4) tbl1 AS k, v) b
ON (a.c1 = b.c1 AND a.c2 = b.k));
Query ID = root_20200218215237_09ca26c0-866f-4789-a403-787826701c49
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-02-18 21:52:46	Starting to launch local task to process map join;	maximum memory = 477626368
2020-02-18 21:52:48	Dump the side-table for tag: 1 with group count: 50 into file: fi

In [16]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM joined_kv;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM joined_kv;
Query ID = root_20200218215314_a215bca6-05c0-4921-ac78-a4fea1b9efd2
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0030, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0030/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0030
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 21:53:22,215 Stage-1 map = 0%,  reduce = 0%
2020-02-18 21:53:28,041 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.8 sec
MapReduce Total cumulative CPU time: 1 seconds 800 msec
Ended Job = job_1582053079980_0030
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_hiv

In [17]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [18]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [19]:
!cat output/000000_0

1,D,3
2,C,0
3,D,1
4,D,5
5,C,2
6,A,4
7,B,5
8,C,5
9,B,4
10,B,3
